# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 9:41AM,243440,12,8360368,LF of America Corp.,Released
1,Aug 4 2022 9:39AM,243439,10,Enova-10419,Emerginnova,Released
2,Aug 4 2022 9:36AM,243438,10,0085923941,ISDIN Corporation,Released
3,Aug 4 2022 9:36AM,243438,10,0085923435,ISDIN Corporation,Released
4,Aug 4 2022 9:36AM,243438,10,0085923606,ISDIN Corporation,Released
5,Aug 4 2022 9:36AM,243438,10,0085923930,ISDIN Corporation,Released
6,Aug 4 2022 9:36AM,243438,10,0085923934,ISDIN Corporation,Released
7,Aug 4 2022 9:36AM,243438,10,0085923939,ISDIN Corporation,Released
8,Aug 4 2022 9:36AM,243438,10,0085923940,ISDIN Corporation,Released
9,Aug 4 2022 9:36AM,243438,10,0085923210,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,243435,Released,15
20,243436,Released,1
21,243438,Released,67
22,243439,Released,1
23,243440,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243435,NaN,15.0
243436,NaN,1.0
243438,NaN,67.0
243439,NaN,1.0
243440,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243355,2.0,1.0
243396,0.0,5.0
243403,9.0,4.0
243414,0.0,1.0
243415,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243355,2,1
243396,0,5
243403,9,4
243414,0,1
243415,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,0,5
2,243403,9,4
3,243414,0,1
4,243415,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243355,2,1
1,243396,,5
2,243403,9,4
3,243414,,1
4,243415,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.
1,Aug 4 2022 9:39AM,243439,10,Emerginnova
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation
69,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC"
70,Aug 4 2022 9:25AM,243435,10,Bio-PRF
85,Aug 4 2022 9:24AM,243434,12,Hush Hush
86,Aug 4 2022 9:24AM,243433,12,Hush Hush
102,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc."
103,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc."
104,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,,1
1,Aug 4 2022 9:39AM,243439,10,Emerginnova,,1
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,,67
3,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",,1
4,Aug 4 2022 9:25AM,243435,10,Bio-PRF,,15
5,Aug 4 2022 9:24AM,243434,12,Hush Hush,,1
6,Aug 4 2022 9:24AM,243433,12,Hush Hush,,16
7,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",1,
8,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",,1
9,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,1,
1,Aug 4 2022 9:39AM,243439,10,Emerginnova,1,
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,67,
3,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",1,
4,Aug 4 2022 9:25AM,243435,10,Bio-PRF,15,
5,Aug 4 2022 9:24AM,243434,12,Hush Hush,1,
6,Aug 4 2022 9:24AM,243433,12,Hush Hush,16,
7,Aug 4 2022 9:08AM,243429,20,"ACI Healthcare USA, Inc.",,1
8,Aug 4 2022 9:04AM,243420,20,"ACI Healthcare USA, Inc.",1,
9,Aug 4 2022 8:57AM,243428,16,Sartorius Bioprocess Solutions,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,1,
1,Aug 4 2022 9:39AM,243439,10,Emerginnova,1,
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,67,
3,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",1,
4,Aug 4 2022 9:25AM,243435,10,Bio-PRF,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,1.0,NaN
1,Aug 4 2022 9:39AM,243439,10,Emerginnova,1.0,NaN
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,67.0,NaN
3,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",1.0,NaN
4,Aug 4 2022 9:25AM,243435,10,Bio-PRF,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 4 2022 9:41AM,243440,12,LF of America Corp.,1.0,0.0
1,Aug 4 2022 9:39AM,243439,10,Emerginnova,1.0,0.0
2,Aug 4 2022 9:36AM,243438,10,ISDIN Corporation,67.0,0.0
3,Aug 4 2022 9:26AM,243436,10,"Nextsource Biotechnology, LLC",1.0,0.0
4,Aug 4 2022 9:25AM,243435,10,Bio-PRF,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2434253,109.0,2.0
12,730307,18.0,0.0
16,243428,2.0,0.0
19,730188,40.0,20.0
20,730252,5.0,10.0
21,243414,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2434253,109.0,2.0
1,12,730307,18.0,0.0
2,16,243428,2.0,0.0
3,19,730188,40.0,20.0
4,20,730252,5.0,10.0
5,21,243414,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,109.0,2.0
1,12,18.0,0.0
2,16,2.0,0.0
3,19,40.0,20.0
4,20,5.0,10.0
5,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,109.0
1,12,Released,18.0
2,16,Released,2.0
3,19,Released,40.0
4,20,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,2.0,0.0,0.0,20.0,10.0,0.0
Released,109.0,18.0,2.0,40.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,2.0,0.0,0.0,20.0,10.0,0.0
1,Released,109.0,18.0,2.0,40.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,2.0,0.0,0.0,20.0,10.0,0.0
1,Released,109.0,18.0,2.0,40.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()